In [ ]:
# default_exp core

# Powerline Detection

> Use Hough Transform to extract powerlines from LiDAR data.

In [ ]:
#hide
from nbdev.showdoc import *
import pathlib
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import laspy as lp

In [ ]:
#export
import numpy as np
from tqdm import tqdm
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
#export
def grid_subsampling(points, voxel_size):
    "Define a function that takes as input an array of points, and a voxel size expressed in meters"
    "points: it is from a LiDAR data; voxel_size: the length of edge"
    nb_vox=np.ceil((np.max(points, axis=0) - np.min(points, axis=0))/voxel_size)
    print(nb_vox.shape)
    a = ((points - np.min(points, axis=0)) // voxel_size).astype(int)
    non_empty_voxel_keys, inverse, nb_pts_per_voxel= np.unique(a, axis=0, return_inverse=True, return_counts=True)
    idx_pts_vox_sorted=np.argsort(inverse)
    voxel_grid={}
    grid_barycenter,grid_candidate_center=[],[]
    last_seen=0
    for idx, vox in enumerate(tqdm(non_empty_voxel_keys)):
        per_voxel = nb_pts_per_voxel[idx]
        num_pvs   = last_seen + per_voxel
        idxs      = idx_pts_vox_sorted[last_seen: num_pvs]
        voxel_grid[tuple(vox)] = points[idxs]
        gbc = np.mean(points[idxs], axis=0)
        grid_barycenter.append(gbc)
        #
        temp = voxel_grid[tuple(vox)]-np.mean(voxel_grid[tuple(vox)],axis=0)
        temp = np.linalg.norm(temp, axis=1).argmin()                                             
        grid_candidate_center.append(voxel_grid[tuple(vox)][temp])
        last_seen += per_voxel
    #
    grid_barycenter       = np.array(grid_barycenter)
    grid_candidate_center = np.array(grid_candidate_center)
    return {'barycenter': grid_barycenter, 'candidate': grid_candidate_center}

In [ ]:
#export
class Arrow3D(FancyArrowPatch):
    "it is a FancyArrowPatch Class, w"
    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0,0),(0,0), *args, **kwargs)
        self._xyz = (x,y,z)
        self._dxdydz = (dx,dy,dz)
    def draw(self, renderer):
        x1,y1,z1 = self._xyz
        dx,dy,dz = self._dxdydz
        x2,y2,z2 = (x1+dx, y1+dy, z1+dz)
                
        xs, ys, zs = proj_transform((x1,x2),(y1,y2),(z1,z2), renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        super().draw(renderer)
#
def _arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
